In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers==4.52.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 108.2 MB/s eta 0:00:0000:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


In [ ]:
from huggingface_hub import login
login()

# Load model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

def generate_text(prompt, max_length=1000, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=False,
        top_k=50,
        top_p=0.95,
        temperature=0.5,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs][0]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

2025-11-26 09:40:36.822647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764150036.990212      46 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764150037.037542      46 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00001-of-00005.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
from langchain.llms.base import LLM
from typing import Any

class CustomHFLLM(LLM):
    def _call(self, prompt: str, stop: Any = None) -> str:
        return generate_text(prompt, max_length=500)

    @property
    def _llm_type(self) -> str:
        return "custom_huggingface"

llm = CustomHFLLM()

# Classification Chain

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
# Chain 1: Task Classification
classify_prompt = PromptTemplate(
    input_variables=["tasks"],
    template=(
        "Classify the following tasks into categories: Urgent, Normal, Optional.\n\n"
        "Tasks:\n{tasks}\n\n"
        "Respond ONLY in JSON like this:\n"
        "{{\n  \"Urgent\": [],\n  \"Normal\": [],\n  \"Optional\": []\n}}\n"
    )
)

classify_chain = LLMChain(
    llm=llm,
    prompt=classify_prompt,
)

/tmp/ipykernel_46/365848651.py:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  classify_chain = LLMChain(


In [ ]:
import json
import re
def extract_json_classify(classify):
    match = re.search(r"```json\s*(\{[\s\S]*?\})\s*```", classify)
    if match:
        json_str = match.group(1)
    else:
        # fallback: extract the last {...} block
        json_candidates = re.findall(r"\{[\s\S]*?\}", classify)
        json_str = json_candidates[-1]

    # Convert string → dict
    classified_tasks_dict = json.loads(json_str)

    # Convert dict → clean JSON text
    classified_tasks = json.dumps(classified_tasks_dict)
    return classified_tasks


# Plan chain

In [ ]:
import json
import re
from langchain import LLMChain, PromptTemplate
# Chain 2: Daily Plan
plan_prompt = PromptTemplate(
    input_variables=["classified_tasks"],
    template=(
        "You are given classified tasks in JSON:\n"
        "{classified_tasks}\n\n"
        "Create a daily plan using ONLY tasks from the input (10 tasks total).\n"
        "Distribute them into three periods: Morning, Afternoon, Evening.\n"
        "***Each period must contain 3–4 tasks to ensure all tasks are included.***\n\n" # MODIFIED CONSTRAINT
        "Respond ONLY in JSON like this:\n"
        "{{\n"
        "  \"Morning\": [],\n"
        "  \"Afternoon\": [],\n"
        "  \"Evening\": []\n"
        "}}\n"
    )
)

plan_chain = LLMChain(
    llm=llm, # 'llm' must be an instantiated Language Model object
    prompt=plan_prompt,
)

#plan = plan_chain.run({"classified_tasks": classified_tasks})

#print(f"\nFeatures:\n{plan.strip()}")

In [ ]:
def extract_json_plan(plan):
    match = re.search(
    r'\{[\s\S]*"Evening":\s*\[[\s\S]*?\][\s\S]*\}',
    plan,
    re.DOTALL | re.IGNORECASE )

    plan_dict = None
    plan_json_str = None
    if match:

        all_json_matches = re.findall(r'(\{[^}]+\})', plan, re.DOTALL)

        if all_json_matches:
            plan_json_str = all_json_matches[-1]
            try:
                # Clean the string (remove excessive whitespace/newlines)
                plan_json_str = re.sub(r'\s+', ' ', plan_json_str).strip()
                # Load the cleaned string into a Python dictionary
                plan_dict = json.loads(plan_json_str)
            except json.JSONDecodeError as e:
                print(f"Error: Extracted text is not valid JSON. {e}")
                print(f"Bad string: {plan_json_str}")
                plan_dict = None
        else:
            print("Error: No JSON dictionary found in LLM output.")

    else:
        print("Error: No JSON structure found in LLM output.")
    return plan_dict

# advice chain

In [ ]:
advice_prompt = PromptTemplate(
    input_variables=["classified_tasks"],
    template=(
        "You are given today's tasks in JSON:\n"
        "{classified_tasks}\n\n"
        "Provide 3 practical productivity tips based ONLY on these tasks.\n\n"
        "Respond ONLY in JSON like this:\n"
        "{{\n"
        "  \"tips\": [\n"
        "    \"Tip 1...\",\n"
        "    \"Tip 2...\",\n"
        "    \"Tip 3...\"\n"
        "  ]\n"
        "}}\n"
    )
)
advice_chain = LLMChain(
    llm=llm,
    prompt=advice_prompt,
)



In [ ]:
def extract_json_advice(advice):
    match = re.search(
    r'\{*"tips":\s*\[[\s\S]*?\][\s\S]*\}',
    advice,
    re.DOTALL | re.IGNORECASE )
    tips_dict = None
    tips_json_str = None
    if match:
        all_json_matches = re.findall(r'(\{[^}]+\})', advice, re.DOTALL)

        if all_json_matches:
            # Take the last JSON block found, as that is the final plan
            tips_json_str = all_json_matches[-1]
            try:
                # Clean the string (remove excessive whitespace/newlines)
                tips_json_str = re.sub(r'\s+', ' ', tips_json_str).strip()
                # Load the cleaned string into a Python dictionary
                tips_dict = json.loads(tips_json_str)

            except json.JSONDecodeError as e:
                print(f"Error: Extracted text is not valid JSON. {e}")
                print(f"Bad string: {tips_json_str}")
                tips_dict = None
        else:
            print("Error: No JSON dictionary found in LLM output.")
    else:
        print("Error: No JSON structure found in LLM output.")
    return tips_dict

# example 1

In [ ]:
task = """
- Finish Python assignment
- Clean the room
- Buy groceries
- Watch a tutorial
- Call mom
- Water the plants
- Exercise for 30 minutes
- Read a chapter of a book
- Organize desk
"""

In [ ]:
classify_out = classify_chain.run({"tasks": task})
classified_json = extract_json_classify(classify_out)


plan_out = plan_chain.run({"classified_tasks": json.dumps(classified_json)})
plan_json = extract_json_plan(plan_out)


advice_out = advice_chain.run({"classified_tasks": json.dumps(classified_json)})
advice_json = extract_json_advice(advice_out)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(classified_json)
print("====================")
print(plan_json)
print("====================")
print(advice_json)

{"Urgent": ["Finish Python assignment", "Call mom"], "Normal": ["Clean the room", "Buy groceries", "Water the plants", "Organize desk"], "Optional": ["Watch a tutorial", "Exercise for 30 minutes", "Read a chapter of a book"]}
{'Morning': ['Finish Python assignment', 'Call mom', 'Clean the room'], 'Afternoon': ['Buy groceries', 'Water the plants', 'Organize desk'], 'Evening': ['Watch a tutorial', 'Exercise for 30 minutes', 'Read a chapter of a book']}
{'tips': ["Prioritize tasks from the 'Urgent' category first, as they require immediate attention.", "Break down larger tasks like 'Clean the room' or 'Organize desk' into smaller, manageable steps to make them less overwhelming.", "Use the 'Optional' tasks as rewards for completing tasks from the 'Urgent' and 'Normal' categories to maintain motivation and productivity."]}


# example 2

In [ ]:
task2 = """
- Review machine learning lesson
- Update your project notes
- Prepare a healthy meal
- Respond to important emails
- Take a 20-minute walk
- Backup important files
- Practice English speaking
- Meditate for 10 minutes
"""
classify_out = classify_chain.run({"tasks": task2})
classified_json = extract_json_classify(classify_out)


plan_out = plan_chain.run({"classified_tasks": json.dumps(classified_json)})
plan_json = extract_json_plan(plan_out)


advice_out = advice_chain.run({"classified_tasks": json.dumps(classified_json)})
advice_json = extract_json_advice(advice_out)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(classified_json)

{"Urgent": ["Respond to important emails", "Backup important files"], "Normal": ["Review machine learning lesson", "Update your project notes", "Prepare a healthy meal", "Plan tomorrow\u2019s tasks"], "Optional": ["Take a 20-minute walk", "Practice English speaking", "Meditate for 10 minutes"]}


# example 3

In [ ]:
task3 = """
- Sort study materials
- Review last lecture notes
- Prepare weekly budget
- Charge all devices
- Check upcoming deadlines
- Practice coding for 1 hour
- Declutter phone gallery
- Drink 2 cups of water
- Set goals for the week
"""
classify_out = classify_chain.run({"tasks": task3})
classified_json = extract_json_classify(classify_out)


plan_out = plan_chain.run({"classified_tasks": json.dumps(classified_json)})
plan_json = extract_json_plan(plan_out)


advice_out = advice_chain.run({"classified_tasks": json.dumps(classified_json)})
advice_json = extract_json_advice(advice_out)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(classified_json)
print("====================")
print(plan_json)
print("====================")
print(advice_json)

{"Urgent": ["Check upcoming deadlines"], "Normal": ["Sort study materials", "Review last lecture notes", "Prepare weekly budget", "Charge all devices", "Practice coding for 1 hour", "Set goals for the week"], "Optional": ["Declutter phone gallery", "Drink 2 cups of water"]}
{'Morning': ['Check upcoming deadlines', 'Sort study materials', 'Review last lecture notes'], 'Afternoon': ['Prepare weekly budget', 'Charge all devices', 'Practice coding for 1 hour'], 'Evening': ['Set goals for the week', 'Declutter phone gallery', 'Drink 2 cups of water']}
{'tips': ["Prioritize tasks by urgency and importance. Start with 'Check upcoming deadlines' under 'Urgent' tasks.", "Break down larger tasks into smaller, manageable steps. For example, 'Sort study materials' can be done category by category.", "Use reminders or alarms to ensure you complete tasks on time. Set a reminder for 'Prepare weekly budget' to ensure it's done before the week starts."]}


In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
NGROK_TOKEN = " " 
API_KEY = "secret12345"

In [ ]:
from fastapi import FastAPI, Request, HTTPException
import uvicorn, threading, time, socket
from pyngrok import ngrok, conf

app = FastAPI()
@app.post("/test")
async def chat(req: Request):
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")

    data = await req.json()
    tasks = data.get("tasks")

    classify_out = classify_chain.run({"tasks": tasks})
    classified_json = extract_json_classify(classify_out)

    # ---- 2) Plan ----
    plan_out = plan_chain.run({"classified_tasks": json.dumps(classified_json)})
    plan_json = extract_json_plan(plan_out)

    # ---- 3) Advice ----
    advice_out = advice_chain.run({"classified_tasks": json.dumps(classified_json)})
    advice_json = extract_json_advice(advice_out)

    # ---- Final Response ----
    return {
        "classified": classified_json,
        "plan": plan_json,
        "advice": advice_json
    }

In [ ]:
def free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = free_port()
conf.get_default().auth_token = NGROK_TOKEN
public_url = ngrok.connect(port).public_url
print("Your public URL:", public_url)

def run(): uvicorn.run(app, host="0.0.0.0", port=port)
threading.Thread(target=run, daemon=True).start()
time.sleep(1)

Your public URL: https://nigel-placid-anabelle.ngrok-free.dev


INFO:     Started server process [47]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:34321 (Press CTRL+C to quit)
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
